In [1]:
import pandas as pd
from math import cos, sin, log, pi, exp
import plotly.graph_objects as go
import numpy as np

In [2]:
from optimization import optimize, golden_section

Тут всякие тестовые функции прихардкодены, на выходе выдают пару: значение функции и значение производной в точке.

Эти же функции есть и *optimization.py*

In [3]:
def f2(x):
    return sin(x) + sin(10 / 3 * x), cos(x) + 10 / 3 * cos(10 / 3 * x)


def f4(x):
    return -(16 * x ** 2 - 24 * x + 5) * exp(-x), (16 * x ** 2 - 56 * x + 29) * exp(-x)


def f5(x):
    return -(1.4 - 3 * x) * sin(18 * x), 3 * (18 * (x - 0.466667) * cos(18 * x) + sin(18 * x))


def f6(x):
    return -(x + sin(x)) * exp(-x ** 2), exp(-x ** 2) * (2 * x ** 2 + 2 * x * sin(x) - cos(x) - 1)


def f7(x):
    return sin(x) + sin(10 / 3 * x) + log(x) - 0.84 * x + 3, 1 / x + cos(x) + 10 / 3 * cos(10 / 3 * x) - 0.84


def f9(x):
    return sin(x) + sin(2 / 3 * x), cos(x) + 2 / 3 * cos(2 / 3 * x)


def f10(x):
    return -x * sin(x), -sin(x) - x * cos(x)


def f11(x):
    return 2 * cos(x) + cos(2 * x), -2 * (sin(x) + sin(2 * x))


def f12(x):
    return sin(x) ** 3 + cos(x) ** 3, 3 * sin(x) * cos(x) * (sin(x) - cos(x))


def f18(x):
    if x <= 3:
        return (x - 2) ** 2, 2 * (x - 2)
    else:
        return 2 * log(x - 2) + 1, 2 / (x - 2)


Далее идут служебная информация по типу: что за функция, на каком интервале ищем минимум, и какой точный миниуму хотим найти
(примеры из http://infinity77.net/global_optimization/test_functions_1d.html)

In [4]:
x1_0 = 0.0
x2_0 = 5.145735
x4_0 = 2.868034
x5_0 = 0.96609
x6_0 = 0.67956
x7_0 = 5.19978
x9_0 = 17.039
x10_0 = 7.9787
x11_0 = 2.09439
x12_0 = pi
x18_0 = 2.0
to_optimize = {
    # 1: (f1, -5, 5, x1_0),
    2: (f2, 2.7, 7.5, x2_0),
    4: (f4, 1.9, 3.9, x4_0),
    5: (f5, 0, 1.2, x5_0),
    6: (f6, -10, 10, x6_0),
    7: (f7, 2.7, 7.5, x7_0),
    9: (f9, 15, 20.4, x9_0),
    10: (f10, 0, 10, x10_0),
    11: (f11, -pi / 2, 2 * pi, x11_0),
    12: (f12, 0, 4, x12_0),
    18: (f18, 0, 6, x18_0)
}

Сам вызов функций поиска минимумов (метода Брента и метода золотого сечения).

In [5]:
data_brent = {}
data_gs = {}
for func_name, (f, a, b, x0) in to_optimize.items():
    raw_data = optimize(f, a, b, x0=x0)
    raw_data_golden = golden_section(f, a, b, x0=x0)

    data = pd.DataFrame(raw_data, columns=["x", "f", "num_iter", "err", "time"])
    data_golden = pd.DataFrame(raw_data_golden, columns=["x", "f", "num_iter", "err", "time"])

    data_brent[func_name] = data
    data_gs[func_name] = data_golden

In [6]:
def plot(fun_num, formula=""):
    global data_brent
    global data_gs
    global to_optimize

    f, a, b, _ = to_optimize[fun_num]
    d1 = data_brent[fun_num]
    d2 = data_gs[fun_num]

    x = np.linspace(a, b, 1000)
    fun = [f(i)[0] for i in x]


    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(
        x=x,
        y=fun,
        mode="lines",
        name=formula
    ))
    fig1.add_trace(go.Scatter(
        x=d1.x,
        y=d1.f,
        mode="markers+text",
        text=d1.num_iter,
        name="step",
        textposition="bottom center"
    ))
    fig1.update_layout(
        title_text=f"f(x) = {formula}: best point at every step",
        xaxis_title="x",
        yaxis_title=formula
    )
    fig1.show()

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=d1.num_iter,
        y=np.log(d1.err),
        mode='markers+lines',
        text=d1.num_iter,
        name="Brent"
    ))
    fig2.add_trace(go.Scatter(
        x=d2.num_iter,
        y=np.log(d2.err),
        mode='markers+lines',
        text=d2.num_iter,
        name="Golden-Section"
    ))
    fig2.update_layout(
        title_text=f"f(x) = {formula}: Brent vs. Golden-section",
        xaxis_title="Number of iterations",
        yaxis_title="log|x - x*|"
    )
    fig2.show()

Далее следуюит графики сравнения реализованного метода Брента с простой реализацией метода золотого сечения. Тонкости реализации (например, критерий остановки) см. в методичке курса методов оптимизации в машинном обучении ВМК МГУ за 2016-ый год (в методичке 2012-ого года используется критерий, который по той или иной причине оказывает менее эффективным).

В последующих картинка приведены два графика для каждой функции: 1-ый график показывает саму функцию и лучшую точку на каждой итерации метода Брента с указанием под ней номера итерации. 2-ой график сравнивает скорость сходимости к "точному" минимому метода Брента с методом золотого сечения.



In [7]:
plot(2, formula="sin(x) + sin(10 / 3 * x)")

Странно, конечно, что первый пример идет на неунимодальной функции, но зато на нем видно, что мы сошлись в минимум (и даже глобальный!), но это просто повезло.

In [8]:
plot(4, formula="-(16 * x ** 2 - 24 * x + 5) * exp(-x)")

Вот эта функция уже унимодальная и в каком-то смысле хорошая, поэтому реализованный метод Брента почти сразу находит нужную точку (хотя и на предыдщем графике потребовалось всего 6 итераций) в то время, как метод золотого сечения все также старательно режет интервалы.

In [9]:
plot(5, formula="-(1.4 - 3 * x) * sin(18 * x)")

Здесь можно заметить, что, казалось бы, метод золотого сечения не сходится, но это не так, потому что он просто сошелся в другой минимум. Все также довольно много итераций нужно для метода золотого сечения по сравнению с методом Брента, который использует информацию о первой производной.

In [10]:
plot(6, formula="-(x + sin(x)) * exp(-x ** 2)")

А вот данная функция сколько-то плохая, так как у нее очень больше пологие области, поэтому нужно больше итераций, чтобы выбраться из них. Стоит заметить, что реализованный метод Брента в случае, когда оптимальная точка, полученная апроксимацией исходной функции параболой, оказалась не очень эффективной, делит отрезки пополам, а не в отношении золотого сечения, поэтому метод Брента должен быстрее вылезать из таких неудобных областей. Хотя запрягает он дольше в самом начале.

В целом, это объясняется тем, что в реализованном методе Брента за начальную "лучшую" точку берется середина интервала, и несколько первых итераций метод просто гонит правую границу (это хорошо видно из первого графика) в то время, как в методе золотого сечения "лучшая" точка постоянно прыгает то налево, то направо.

Тем не менее, реализованный метод сошелся в 4 раза быстрее, чем простой метод золотого сечения.

Вообще тут можно сделать вывод, что метод золотого сечения плюс/минус за одно и то же количество итераций сходится на разной хорошести/паршивости функций, а метод Брента может на одних хороших функциях сойтись очень быстро, а на других, плохих функциях сойтись подольше, но все равно эта скорость намного больше, чем у метода золотого сечения.

In [11]:
# plot(7, formula="sin(x) + sin(10 / 3 * x) + log(x) - 0.84 * x + 3")

In [12]:
# plot(9, formula="sin(x) + sin(2 / 3 * x)")

In [13]:
# plot(10, formula="-x * sin(x)")

In [14]:
# plot(11, formula="2 * cos(x) + cos(2 * x)")

In [15]:

# plot(12, formula="sin(x) ** 3 + cos(x) ** 3")

In [16]:
# plot(18, formula="Piecewise")

Можно сделать вывод, что метод Брента с использованием производной для приближения функции параболой по значениям производной в двух точках дает результат за несоизмеримо меньшее количество шагов итераций в сравнении с наивным методом золотого сечения.

Стоит отдельно отметить, что в обоих реализованных методах оракул вызывается один раз за каждый шаг итерации, однако в методе Брента делается всего один инициализирующий вызов оракула, в то время как в метода золотого сечения --- два.

P.S. оставил только сколько-то интересные примеры, так как *plotly* невероятно нагружает систему, если всю пачку графиков в интерактивном режиме оставлять (покрайней мере *pycharm* сходит с ума, когда много графиков). А так в целом даже удивлен, что так быстро можно свалиться в минимум. Интересная задачка, спасибо.